# Ideas policies table view

In [1]:
import pandas as pd
import numpy as np
from openpyxl.styles import PatternFill

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
from src.model import *
from src.loader_tools import *

In [3]:
IDEAS: dict[str, Idea] = load_ideas([
    '../data/processed/gypsy-1-36-1/00_basic_ideas.yaml', 
    '../data/processed/gypsy-1-36-1/00_flogi_ideas.yaml', 
])

POLICIES: dict[str, Policy] = load_policies([
    '../data/processed/gypsy-1-36-1/Idea_Variation_policies.yaml'
])

In [4]:
RELIGIOUS_IDEA_NAMES = {'religious_ideas', 'anglican0', 'animist0', 'buddhism0', 'cathar0', 'catholic0', 'confucian0', 'coptic0', 'dreamtime0', 'fetishist0', 'hellenic0', 'hindu0', 'hussite0', 'ibadi0', 'inti0', 'jewish0', 'manichean0', 'mesoamerican0', 'nahuatl0', 'norse0', 'orthodox0', 'protestant0', 'reformed0', 'romuva0', 'shia0', 'shinto0', 'slavic0', 'sunni0', 'suomi0', 'tengri0', 'totemist0', 'zoroastrian0'}
GOVERNMENT_IDEA_NAMES = {'dictatorship0', 'horde0', 'monarchy0', 'republic0', 'theocracy0'}

def get_idea_type(idea_name: str) -> str:
    if idea_name in IDEAS:
        return IDEAS[idea_name].type
    elif idea_name == 'religious_ideas':
        return 'ADM'
    elif idea_name == 'government_ideas':
        return 'ADM'
    else:
        raise ValueError(f'Unknown idea name: {idea_name}')

GROUPED_IDEA_NAMES = [idea.name for idea in IDEAS.values() if idea.name not in RELIGIOUS_IDEA_NAMES and idea.name not in GOVERNMENT_IDEA_NAMES] + ['religious_ideas', 'government_ideas']
GROUPED_IDEA_NAMES = sorted(GROUPED_IDEA_NAMES)
GROUPED_IDEA_NAMES = sorted(GROUPED_IDEA_NAMES, key=lambda x: (get_idea_type(x), x))

In [5]:
POLICY_LIB = {}
for policy in POLICIES.values():
    req0, req1 = policy.req
    for req0_1 in req0:
        for req1_1 in req1:
            policy_id = tuple(sorted([req0_1, req1_1]))
            POLICY_LIB[policy_id] = policy

In [6]:
def color_policy(v, color):
    if v.startswith('adm') or v.startswith('ADM'):
        return 'background-color: green;'
        # return 'color: green;'
    elif v.startswith('dip') or v.startswith('DIP'):
        return 'background-color: blue;'
        # return 'color: blue;'
    elif v.startswith('mil') or v.startswith('MIL'):
        return 'background-color: red;'
        # return 'color: red;'
    else:
        return ''
    

In [7]:
def replace_idea_name(idea_name):
    if idea_name in RELIGIOUS_IDEA_NAMES:
        return 'religious_ideas'
    elif idea_name in GOVERNMENT_IDEA_NAMES:
        return 'government_ideas'
    else:
        return idea_name

policy_table = pd.DataFrame(
    index=GROUPED_IDEA_NAMES,
    columns=GROUPED_IDEA_NAMES,
)
policy_table = policy_table.fillna('-')
for policy in POLICIES.values():
    req1, req2 = policy.req
    policy_text = f'{policy.type} {policy.name}\n'
    for key, value in policy.effect.items():
        policy_text += f'{key}: {value}\n'
    for idea_a_name in req1:
        for idea_b_name in req2:
            idea_a_name = replace_idea_name(idea_a_name)
            idea_b_name = replace_idea_name(idea_b_name)
            if policy_table.loc[idea_a_name, idea_b_name] == '-':
                policy_table.loc[idea_a_name, idea_b_name] = policy_text
                policy_table.loc[idea_b_name, idea_a_name] = policy_text
            else:
                if policy_table.loc[idea_a_name, idea_b_name] != policy_text:
                   print(f'Conflict: {policy_table.loc[idea_a_name, idea_b_name].split()[1]} vs {policy.name} for {idea_a_name} and {idea_b_name}')
            

# policy_table.fillna('', inplace=True)
policy_table.style.map(color_policy, color='red')



Conflict: idea_variation_act_152 vs idea_variation_act_23 for jurisprudence0 and propaganda0
Conflict: idea_variation_act_160 vs idea_variation_act_437 for war_production0 and propaganda0


,administrative_ideas,centralisation0,decentralisation0,development0,economic_ideas,expansion_ideas,government_ideas,humanist_ideas,innovativeness_ideas,jurisprudence0,religious_ideas,state_administration0,strongfem0,strongman0,assimilation0,colonial_empire0,dynasty0,exploration_ideas,fleet_base0,galley0,heavy_ship0,imperial_ambition0,imperialism0,influence_ideas,light_ship0,maritime_ideas,nationalism0,propaganda0,society0,spy_ideas,trade_ideas,conscription0,defensive0,fire0,fortress0,general_staff0,mercenary0,militarism0,offensive0,quality0,quantity0,shock0,standing_army0,tactical0,war_production0,weapon_quality0
administrative_ideas,-,-,-,-,-,-,-,-,-,-,-,-,-,-,DIP idea_variation_act_186 diplomatic_annexation_cost: 0.15,ADM idea_variation_act_111 state_maintenance_modifier: 0.5 yearly_corruption: 0.05,DIP idea_variation_act_574 diplomatic_reputation: 1 diplomats: 1,DIP idea_variation_act_573 colonist_placement_chance: 0.1 global_colonial_growth: 10,ADM idea_variation_act_127 governing_capacity_modifier: 0.25,DIP idea_variation_act_586 global_ship_repair: 0.2 trade_efficiency: 0.2,DIP idea_variation_act_586 global_ship_repair: 0.2 trade_efficiency: 0.2,ADM idea_variation_act_200 governing_capacity_modifier: 0.25,ADM idea_variation_act_200 governing_capacity_modifier: 0.25,DIP idea_variation_act_581 diplomatic_annexation_cost: 0.15,DIP idea_variation_act_586 global_ship_repair: 0.2 trade_efficiency: 0.2,DIP idea_variation_act_557 navy_tradition: 2,ADM idea_variation_act_74 global_unrest: 2,ADM idea_variation_act_144 inflation_reduction: 0.02 interest: 1,ADM idea_variation_act_287 state_maintenance_modifier: 0.5 yearly_corruption: 0.05,ADM idea_variation_act_564 global_trade_goods_size_modifier: 0.15,DIP idea_variation_act_593 trade_efficiency: 0.25,ADM idea_variation_act_315 global_regiment_cost: 0.15 global_regiment_recruit_speed: 0.3,ADM idea_variation_act_567 defensiveness: 0.1 yearly_corruption: 0.1,MIL idea_variation_act_479 fire_damage: 0.075,MIL idea_variation_act_108 fire_damage_received: 0.05,MIL idea_variation_act_84 cav_to_inf_ratio: 0.1 cavalry_power: 0.1,ADM idea_variation_act_305 merc_maintenance_modifier: 0.15 mercenary_cost: 0.15,ADM idea_variation_act_376 land_morale: 0.15,MIL idea_variation_act_570 shock_damage: 0.075,ADM idea_variation_act_631 state_maintenance_modifier: 0.5 yearly_corruption: 0.05,ADM idea_variation_act_629 manpower_recovery_speed: 0.2,ADM idea_variation_act_449 shock_damage: 0.075,ADM idea_variation_act_87 land_maintenance_modifier: 0.2,MIL idea_variation_act_405 fire_damage: 0.075,ADM idea_variation_act_105 inflation_reduction: 0.02 interest: 1,ADM idea_variation_act_324 land_attrition: 0.1 reinforce_speed: 0.2
centralisation0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,DIP idea_variation_act_512 culture_conversion_cost: 0.2,DIP idea_variation_act_511 global_tariffs: 0.25,ADM idea_variation_act_528 development_cost: 0.1,DIP idea_variation_act_504 colonists: 1,ADM idea_variation_act_515 range: 0.5 trade_efficiency: 0.1,DIP idea_variation_act_537 naval_morale: 0.15,DIP idea_variation_act_537 naval_morale: 0.15,ADM idea_variation_act_516 build_cost: 0.15,ADM idea_variation_act_516 build_cost: 0.15,ADM idea_variation_act_529 build_cost: 0.15,DIP idea_variation_act_537 naval_morale: 0.15,DIP idea_variation_act_505 naval_forcelimit_modifier: 0.15 naval_maintenance_modifier: 0.15,ADM idea_variation_act_517 global_tax_modifier: 0.15 production_efficiency: 0.15,DIP idea_variation_act_514 development_cost: 0.1,ADM idea_variation_act_513 inflation_reduction: 0.02 interest: 1,DIP idea_variation_act_527 province_warscore_cost: 0.2,DIP idea_variation_act_532 development_cost: 0.1,MIL idea_variation_act_550 land_morale: 0.15,MIL idea_variation_act_531 land_morale: 0.15,ADM idea_variation_act_496 global_trade_goods_size_modifier: 0.15,MIL idea_variation_act_552 shock_damage_received: 0.05,ADM idea_variation_act_548 global_manpower_modifier: 0.25,ADM idea_variation_act_520 mercenary_manpower: 0.5,ADM idea_variation_act_55

In [8]:
with pd.ExcelWriter('../export/dev_policy_table.xlsx', engine="openpyxl") as writer:
    sheet_name = 'policy_table'
    policy_table.to_excel(writer, sheet_name=sheet_name, index=True, header=True)
    sheet = writer.sheets[sheet_name]
    
    column_range = sheet['B:CE']
    
    for column in column_range:
        for cell in column[1:]:
            if cell.value.startswith('adm') or cell.value.startswith('ADM'):
                cell.fill = PatternFill("solid", fgColor="5cb800")
            elif cell.value.startswith('dip') or cell.value.startswith('DIP'):
                cell.fill = PatternFill("solid", fgColor="00b0f0")
            elif cell.value.startswith('mil') or cell.value.startswith('MIL'):
                cell.fill = PatternFill("solid", fgColor="ff2800")


AttributeError: 'NoneType' object has no attribute 'startswith'